# Raphtory Python Client - LOTR DEMO

In this demo we will
* Setup the Python Raphtory client 
* Run an algorithm from python and save the result to a file
* Create a Graph from data in Raphtory
* Pull results from an algorithm in Raphtory that was saved to a pulsar topic 
* Add these results to the Graph
* Visualise the graph with pymotif

### Setup

Please ensure you are running the `raphtory-example-lotr`. 

Either the `SimpleRunner` or `PulsarOutputRunner`

From there please obtain the deployment id, e.g. `raphtory_1172949031` from

```
17:32:15.975 [main] INFO  com.raphtory.client.GraphDeployment - Created Graph object with deployment ID 'raphtory_1172949031'.
```

First we setup the various python libraries we will need

You can install the python client via `pip install raphtory-client`

In [ ]:
import raphtoryclient
from pymotif import Motif

#### Create Client

Now we create a client.

This is responsible for two things, 

1) it will be able to interface with the raphtory engine to run algorithms as if we are running via scala

2) be able to retrieve data directly from the pulsar interface

In [ ]:
raphtory = raphtoryclient.client(raphtory_deployment_id="raphtory_1172949031")

### Running a algorithm 

To run an algorithm we must first import the java path it belongs to. 
In our case we want to run the Connected components algorithms. 

We will import this into our local client, so that our code is able to call them.

In [ ]:
raphtory.java_import("com.raphtory.algorithms.generic.ConnectedComponents")
connectedComponentsAlgorithm = raphtory.java().ConnectedComponents

We also want to write the result of this to a file, so we can use the FileOutputFormat class to do so. 

We first import the class as follows

In [ ]:
raphtory.java_import("com.raphtory.output.FileOutputFormat")
fileOutputFormat = raphtory.java().FileOutputFormat

In this example, 

- we are running the algorithm at time 32674 (which is the latest point in our graph). 
- with the `past()` function, which will ensure the graph takes into account all the history from and including 32674
- We 'execute' the connected components algorithm 
- Finally we write this to a file

We can now run the algorithm as follows. 

In [ ]:
queryHandler = raphtory.graph\
    .at(32674)\
    .past()\
    .execute(connectedComponentsAlgorithm())\
    .writeTo(fileOutputFormat.apply("/tmp/pythonCC"))

If we now investigate the `/tmp/pythonCC` file we will see that 
it contains our results. 

You can check the status of this algorithm 
in the java terminal where your raphtory instance is running or you can run the following.

In [ ]:
queryHandler.isJobDone()

## Exporting data 

Previously we ran an algorithm that wrote our data to a file. 

Now we will look at exporting results from an algorithm that was saved to Pulsar topics. 

This portion of the tutorial requires that you are running the `raphtory-example-lotr/.../PulsarOutputRunner`

We will create the readers which read from topics.

If the connection fails, the code with automatically retry.

This can occur when you have not closed previous pulsar connections.

In this case we are reading the topics: EdgeList, PageRank

Note: Prior to this you should have already run these algorithms in Raphtory.

In [ ]:
edgeListReader = raphtory.createReader("EdgeList", subscription_name='edgelist_reader')
pageRankReader = raphtory.createReader("PageRank", subscription_name='pagerank_reader')

### Obtain dataframes

Now we will run the getDataframe function to retrieve results as dataframes.

In [ ]:
df_edge = raphtory.getDataframe(edgeListReader)

In [ ]:
df_edge

In [ ]:
df_page = raphtory.getDataframe(pageRankReader)

In [ ]:
df_page

### Create a graph

Next we create a graph by pulling the edge list from Raphtory.

In this case we would like to create a graph from the LOTR dataset.

So we run the `createGraphFromEdgeList` method on the `EdgeList` dataframe.

In [ ]:
G = raphtory.createGraphFromEdgeList(df_edge, isMultiGraph=False)
G.number_of_nodes(), G.number_of_edges()

### Adding properties to our graph
Now we merge these as node properties into our graph

In [ ]:
raphtory.add_node_attributes(G, [df_page], ['PageRank'])

### Visualisation

Finally we plot the graph with an open source visualisation tool .

In [ ]:
motif_nx = Motif(nx_graph=G, title='NetworkX')
motif_nx.plot()